In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import Point, LineString
from shapely.ops import split

In [3]:
#Read in routes feature class as shapefile
gdfRoutes = gpd.read_file('..\\data\\processed\\Routes.shp')

In [4]:
#Columns to drop to keep things tidy
drop_cols = ['Facility N', 'Address', 'City', 'County Nam',
             'Zip', 'Latitude', 'Longitude', 'Regulated', 
             'Allowable', 'Total Wast']

In [6]:
#Copy routes geodataframe and update geometry to start points
gdfStart = gdfRoutes.copy(deep=True)
gdfStart['geometry'] = gdfRoutes['geometry'].apply(lambda x: Point(x.coords[0]))
gdfStart.drop(columns=drop_cols,axis=1,inplace=True)
gdfStart.to_file('../scratch/starts.shp')

In [7]:
#Copy routes geodataframe and update geometry to start points
gdfEnd = gdfRoutes.copy(deep=True)
gdfEnd['geometry'] = gdfRoutes['geometry'].apply(lambda x: Point(x.coords[-1]))
gdfEnd.drop(columns=drop_cols,axis=1,inplace=True)
gdfEnd.to_file('../scratch/ends.shp')

Create junctions along route segments where upstream routes join them - done by iterating through each route feature and splitting it by the set of end points. 

In [8]:
#Combine endpoints into a single multipoint object
ends = gdfEnd.geometry.unary_union

In [9]:
#Create a geoseries of split routes (geometry collections)
theSplits = gdfRoutes.geometry.apply(lambda x: split(x,ends))

* Create a feature class of all segments (routes split at junctions)

In [35]:
#Create lists to fill
links = []
startIDs = []; endIDs = []
biogas = []
geom = []
#Iterate and add items to the list
for index, row in gdfRoutes.iterrows():
    #Iterate through split segments in the geometry collection
    for line in theSplits[index].geoms:
        #Add items to the list
        links.append(row['index'])
        biogas.append(row['Biogas P_1'])
        geom.append(line)

#Construct an output geodataframe
gdfSegments = gpd.GeoDataFrame(pd.DataFrame({'route_id':links}),
                               geometry = geom, crs = gdfRoutes.crs)

#Add the index as a unique segment ID  
gdfSegments['edge_ID'] = gdfSegments.index

* Create feature classes from from and to nodes

In [64]:
#Construct a gdf of segement start and end points
gdfToNodes = gdfSegments.copy(deep=True)
gdfToNodes['geometry'] = gdfToNodes['geometry'].apply(lambda x: Point(x.coords[0]))
gdfToNodes['to_id'] = gdfToNodes.index

gdfFromNodes = gdfSegments.copy(deep=True)
gdfFromNodes['geometry'] = gdfFromNodes['geometry'].apply(lambda x: Point(x.coords[-1]))
gdfFromNodes['from_id'] = gdfFromNodes.index

In [68]:
#Spatially join the above geodataframes
gdfNodes = gpd.sjoin(left_df=gdfToNodes, right_df=gdfFromNodes,how='right')
#gdfNodes.to_file('../scratch/nodes.shp')
gdfNodes.head()

,index_left,route_id_x,edge_ID_x,to_id,route_id_y,geometry,edge_ID_y,from_id
0,1.0,179.0,1.0,1.0,179,POINT (1582309.906 -310281.512),0,0
1,2.0,179.0,2.0,2.0,179,POINT (1582309.906 -308768.307),1,1
2,3.0,179.0,3.0,3.0,179,POINT (1582309.906 -308263.906),2,2
3,4.0,179.0,4.0,4.0,179,POINT (1582309.906 -307255.102),3,3
4,5.0,179.0,5.0,5.0,179,POINT (1582814.308 -305741.898),4,4


In [95]:
gdfSegments.head()

,route_id,geometry,edge_ID
0,179,"LINESTRING (1582814.308 -311794.717, 1582814.3...",0
1,179,"LINESTRING (1582309.906 -310281.512, 1582309.9...",1
2,179,"LINESTRING (1582309.906 -308768.307, 1582309.9...",2
3,179,"LINESTRING (1582309.906 -308263.906, 1582309.9...",3
4,179,"LINESTRING (1582309.906 -307255.102, 1582814.3...",4


In [110]:
gdfOut = gdfSegments.merge(gdfNodes[['to_id','from_id']],left_on='edge_ID', right_on='from_id')
gdfOut.head()

,route_id,geometry,edge_ID,to_id,from_id
0,179,"LINESTRING (1582814.308 -311794.717, 1582814.3...",0,1.0,0
1,179,"LINESTRING (1582309.906 -310281.512, 1582309.9...",1,2.0,1
2,179,"LINESTRING (1582309.906 -308768.307, 1582309.9...",2,3.0,2
3,179,"LINESTRING (1582309.906 -308263.906, 1582309.9...",3,4.0,3
4,179,"LINESTRING (1582309.906 -307255.102, 1582814.3...",4,5.0,4


In [111]:
#Join BG data
gdfOut2 = gdfOut.merge(gdfRoutes[['index','Biogas P_1']],left_on='from_id',right_on='index',how='left')
gdfOut2.head()

,route_id,geometry,edge_ID,to_id,from_id,index,Biogas P_1
0,179,"LINESTRING (1582814.308 -311794.717, 1582814.3...",0,1.0,0,0.0,3.937494e+06
1,179,"LINESTRING (1582309.906 -310281.512, 1582309.9...",1,2.0,1,1.0,2.279149e+06
2,179,"LINESTRING (1582309.906 -308768.307, 1582309.9...",2,3.0,2,2.0,5.509951e+06
3,179,"LINESTRING (1582309.906 -308263.906, 1582309.9...",3,4.0,3,3.0,2.780021e+06
4,179,"LINESTRING (1582309.906 -307255.102, 1582814.3...",4,5.0,4,4.0,2.780021e+06


In [114]:
gdfOut2[['edge_ID','from_id','to_id','route_id','Biogas P_1','geometry']].to_file('../scratch/featlines.shp')

* Join the from and to node IDs to each segment

In [12]:
#Find the start point corresponding with this value
def getNodeID(line_segment,start=True):
    #Get the start or end point
    if start: 
        gdf = gdfFromNodes #Set the search geodataframe as the one of start points
        idx = 0        #Set the index to the first point of the line segment
        col = 'from_id'
    else:
        gdf = gdfToNodes   #Set the search geodataframe as the one of end points
        idx = -1       #Set the index to the last point of the line segment
        col = 'to_id'
    #Create the search point object, buffered 5 meters
    theSearchPoly = Point(line_segment.coords[idx]).buffer(5)
    #Select all records found within the buffered search points
    gdfSelect = gdf[gdf.geometry.within(theSearchPoly)]
    #Return the index value of the returned record
    if gdfSelect.shape[0]>0: return gdfSelect[col].values[0]
    else: return -1

#Add start node ids to edges
gdfSegments['FromID'] = gdfSegments['geometry'].apply(getNodeID)
#Add end node ids to edges
gdfSegments['ToID'] = gdfSegments['geometry'].apply(lambda x: getNodeID(x,start=False))

In [62]:
#Write to file
gdfSegments.to_file('../scratch/LineSegements.shp')
gdfFromNodes.to_file('../scratch/FromNodes.shp')
gdfToNodes.to_file('../scratch/ToNodes.shp')

---
1. Join segment IDs to from nodes


In [63]:
gdfNodes = gpd.sjoin(left_df=gdfFromNodes,right_df=gdfToNodes,how='right')
gdfNodes.rename({'edge_ID_left':'fromID','edge_ID_right':'toID'},inplace=True)
gdfNodes.head()

,index_left,route_id_x,edge_ID_x,from_id,route_id_y,geometry,edge_ID_y,to_id
0,1.0,179.0,1.0,1.0,179,POINT (1582309.906 -310281.512),0,0
1,2.0,179.0,2.0,2.0,179,POINT (1582309.906 -308768.307),1,1
2,3.0,179.0,3.0,3.0,179,POINT (1582309.906 -308263.906),2,2
3,4.0,179.0,4.0,4.0,179,POINT (1582309.906 -307255.102),3,3
4,5.0,179.0,5.0,5.0,179,POINT (1582814.308 -305741.898),4,4


In [40]:
gdfNodes.to_file('../scratch/Nodes.shp')

In [51]:
df = pd.merge(left=gdfFromNodes,left_on='edge_ID',
              right=gdfToNodes,right_on='edge_ID',
              how='outer')
lines = [LineString(pts) for pts in zip(df.geometry_x,df.geometry_y) ]

In [59]:
gdf = gpd.GeoDataFrame(df,geometry=lines,crs=gdfRoutes.crs)
gdf[['route_id_x','edge_ID','from_id','to_id','geometry']].to_file('../scratch/lines.shp')

In [42]:
gdf.head()

,route_id,geometry,edge_ID,to_id
0,179,POINT (1582309.906 -310281.512),0,0
1,179,POINT (1582309.906 -308768.307),1,1
2,179,POINT (1582309.906 -308263.906),2,2
3,179,POINT (1582309.906 -307255.102),3,3
4,179,POINT (1582814.308 -305741.898),4,4


In [36]:
gdfSegments

,route_id,geometry,edge_ID
0,179,"LINESTRING (1582814.308 -311794.717, 1582814.3...",0
1,179,"LINESTRING (1582309.906 -310281.512, 1582309.9...",1
2,179,"LINESTRING (1582309.906 -308768.307, 1582309.9...",2
3,179,"LINESTRING (1582309.906 -308263.906, 1582309.9...",3
4,179,"LINESTRING (1582309.906 -307255.102, 1582814.3...",4
...,...,...,...
2565,715,"LINESTRING (1640316.091 -291618.653, 1640316.0...",2565
2566,217,"LINESTRING (1722029.151 -118104.500, 1722533.5...",2566
2567,1968,"LINESTRING (1606521.183 -297167.070, 1606521.1...",2567
2568,1463,"LINESTRING (1510684.878 -155934.621, 1510684.8...",2568
